In [1]:
import syft as sy
import numpy as np

In [6]:
from scipy.optimize import minimize_scalar
from typing import Tuple, Callable

In [10]:
def _get_fake_rdp_func(constant: int) -> Callable:
    def func(alpha: float) -> float:
        return alpha * constant

    return func

In [11]:
def _get_alpha_search_function(rdp_compose_func: Callable) -> Callable:
    # if len(self.deltas) > 0:
    # delta = np.max(self.deltas)
    # else:
    log_delta = np.log(1e-6)

    def fun(alpha: float) -> float:  # the input is the RDP's \alpha
        if alpha <= 1:
            return np.inf
        else:
            alpha_minus_1 = alpha - 1
            return np.maximum(
                rdp_compose_func(alpha)
                + np.log(alpha_minus_1 / alpha)
                - (log_delta + np.log(alpha)) / alpha_minus_1,
                0,
            )

    return fun

In [41]:
from tqdm import tqdm

In [12]:
def _get_optimal_alpha_for_constant(constant: int = 3
) -> Tuple[np.ndarray, Callable]:
    f = _get_fake_rdp_func(constant=constant)
    f2 = _get_alpha_search_function(rdp_compose_func=f)
    results = minimize_scalar(
        f2, method="Brent", bracket=(1, 2), bounds=[1, np.inf]
    )
    return results.x, results.fun

In [42]:
def _increase_max_cache(new_size: int, _cache_constant2epsilon) -> None:
    new_entries = []
    current_size = len(_cache_constant2epsilon)
    new_alphas = []
    for i in tqdm(range(new_size - current_size)):
        alph, eps = _get_optimal_alpha_for_constant(constant=i + 1 + current_size)
        new_entries.append(eps)
        new_alphas.append(alph)
    _cache_constant2epsilon = np.concatenate(
        [_cache_constant2epsilon, np.array(new_entries)]
    )
    return _cache_constant2epsilon

In [43]:
cache = np.array([], dtype=np.float64)

In [44]:
cache = _increase_max_cache(300_000, _cache_constant2epsilon=cache)

  0%|                                                                                                                                                              | 0/300000 [00:00<?, ?it/s]/home/ruchi/anaconda3/envs/syft_env/lib/python3.9/site-packages/scipy/optimize/_optimize.py:2782: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300000/300000 [01:17<00:00, 3857.95it/s]


In [45]:
cache.shape

(300000,)

In [47]:
cache[25]

62.28307625241597

In [48]:
cache[20_000]

21047.678272683446

In [49]:
from time import time

t0 = time()
np.savetxt("cache300k.txt", cache)
tf = time() - t0
print(tf)

0.45233607292175293


In [50]:
t0 = time()
restored_cache = np.loadtxt("cache300k.txt")
tf = time() - t0
print(tf)

0.34182310104370117


In [51]:
restored_cache.dtype

dtype('float64')

In [14]:
_get_optimal_alpha_for_constant(constant=1)

(4.508496357814772, 7.766216625311721)

In [37]:
recreated_cache = np.loadtxt("cache1000.txt", dtype=float)

In [39]:
(recreated_cache == cache).all()

True

In [15]:
sigma = 1e5
rdp_constant = np.square(25000 - 500)/(2* (sigma**2))

In [16]:
rdp_constant

0.0300125

In [17]:
_get_optimal_alpha_for_constant(constant=rdp_constant)

/home/ruchi/anaconda3/envs/syft_env/lib/python3.9/site-packages/scipy/optimize/_optimize.py:2782: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


(19.987619840572194, 1.1184184678426403)

In [21]:
# for 1B
scale=1
min_val=750_000/scale 
max_val=20_000_000/scale
sigma_1b = 1e11

rdp_constant_1b = np.square(max_val-min_val)/(2*(sigma_1b**2))
rdp_constant_1b

1.8528125e-08